In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import itertools as it
import datetime
import math

df = pd.read_csv('glicko_dat_1.csv', low_memory = False)

In [2]:
df = df[(df['Year'] == 2017)]
df.head(10)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day
0,413,104.0,1.0,1.0,33.0,NaN,22.00,1.0,2/23/2017,40.92 km/h,0.0,189 km,20,2017,2,23
1,414,58.0,1.0,1.0,33.0,NaN,22.00,2.0,2/24/2017,44.1 km/h,0.0,153 km,20,2017,2,24
2,415,116.0,1.0,1.0,33.0,NaN,22.00,3.0,2/25/2017,40.71 km/h,120.0,186 km,20,2017,2,25
3,416,62.0,1.0,1.0,33.0,NaN,22.00,4.0,2/26/2017,46.86 km/h,0.0,143 km,20,2017,2,26
4,417,102.0,1.0,1.0,33.0,3.0,22.00,-1.0,2/26/2017,-,0.0,0 km,20,2017,2,26
5,418,24.0,1.0,1.0,33.0,NaN,11.00,1.0,3/5/2017,43.95 km/h,0.0,148.5 km,4,2017,3,5
6,419,117.0,1.0,1.0,33.0,NaN,11.00,2.0,3/6/2017,44.83 km/h,0.0,195 km,4,2017,3,6
7,420,48.0,1.0,1.0,33.0,NaN,11.00,3.0,3/7/2017,42.03 km/h,0.0,190 km,4,2017,3,7
8,421,30.0,1.0,1.0,33.0,NaN,11.00,4.0,3/8/2017,40.18 km/h,0.0,14.5 km,4,2017,3,8
9,422,37.0,1.0,1.0,33.0,NaN,11.00,5.0,3/9/2017,42.21 km/h,0.0,199.5 km,4,2017,3,9


In [3]:
##### RATE FUNCTION DON'T TOUCH!!!! ######
Q = math.log(10) / 400
def reduce_impact(rd):
        """The original form is `g(RD)`. This function reduces the impact of
        games as a function of an opponent's RD.
        """
        return (1 + (3 * (Q ** 2) * rd ** 2) / math.pi ** 2) ** -0.5

def expect_score(elo, other_elo, other_impact):
        return 1 / (1 + 10 ** (other_impact * (elo - other_elo) / -400.))

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros
def onelistmaker(n):
    listofones = [1] * n
    return listofones
# elos = list(combined['old_elo'])
# rds = list(combined['old_rd'])
def rate(old_elos, old_rds): # assuming sorted by rank
        d_square_inv = 0
        difference = 0
        new_elos = []
        new_rds = []

        for i in range(len(old_elos)): # do this for every rider, i denotes rider that's being ranked
            if i < len(old_elos):
                d_square_inv = 0
                difference = 0
                denom = 0


                better = old_elos[:i]
                worse = old_elos[i+1:]
                other_elos = better + worse
                # now create a list of e.g. 3x0, rest ones if score 4th place, j denotes every other rider
                actual_score = zerolistmaker(len(better)) + onelistmaker(len(worse))
                for j in range(len(other_elos)):
                    impact = reduce_impact(other_elos[j])
                    expected_score = expect_score(old_elos[i], other_elos[j], impact)
                    difference += impact * (actual_score[j] - expected_score)
                    d_square_inv += (expected_score * (1 - expected_score) * (Q ** 2) * (impact ** 2))

                denom = 1/(old_rds[i] ** 2) + d_square_inv
                mu = old_elos[i]  + Q / denom * difference
                sigma = math.sqrt(1. / denom)
                new_elos.append(mu)
                new_rds.append(sigma)
            else:
                d_square_inv = 0
                difference = 0
                denom = 0

                better = old_elos
                worse = []
                other_elos = better + worse
                # now create a list of e.g. 3x0, rest ones if score 4th place, j denotes every other rider
                actual_score = zerolistmaker(len(better)) + onelistmaker(len(worse))
                for j in range(len(other_elos)):
                    impact = reduce_impact(other_elos[j])
                    expected_score = expect_score(old_elos[i], other_elos[j], impact)
                    difference += impact * (actual_score[j] - expected_score)
                    d_square_inv += (expected_score * (1 - expected_score) * (Q ** 2) * (impact ** 2))
                denom = 1/(old_rds[i] ** 2) + d_square_inv
                mu = old_elos[i]  + Q / denom * difference
                sigma = math.sqrt(1. / denom)
                new_elos.append(mu)
                new_rds.append(sigma)


        
        return(new_elos, new_rds)

In [4]:
# ONE GC SORTED BASED ON RANK
dat = df[(df['Stage'] == -1)].copy()
dat = dat[(dat['Name'] == "11.00")].sort_values("Pnt", ascending=False) # RNK 0 == DNS/DNS
#dat = dat[(dat['Rnk'] != 0.0)]
dat['old_elo'] = 1500
dat['new_elo'] = 0
dat['old_rd'] = 350
dat['new_rd'] = 0

Q = math.log(10) / 400
#dat['g_rd'] = (1 + (3 * (Q ** 2) * dat['old_rd'] ** 2) / math.pi ** 2) ** -0.5
dat.head(20)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
2082,18043,8.0,80.0,58.0,29.0,250.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
29568,204503,1.0,1270.0,7.0,34.0,190.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
1274,10864,3.0,55.0,19.0,30.0,160.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3946,31378,49.0,140.0,1.0,29.0,140.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
13967,101165,41.0,491.0,19.0,24.0,120.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
5293,41516,5.0,190.0,75.0,27.0,110.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3213,26447,42.0,121.0,95.0,28.0,100.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16859,120903,47.0,582.0,103.0,25.0,90.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
15937,113956,32.0,552.0,28.0,24.0,80.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16013,114485,178.0,554.0,52.0,28.0,70.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0


In [5]:
# FILTER 2 BEST RANKED RIDERS
first = dat.groupby('Team').nth(0)
second = dat.groupby('Team').nth(1)

combined = first.append(second)
combined = combined.reset_index().sort_values('Pnt', ascending=False)
rada = np.arange(1, (len(combined.Rnk)+1))
combined.Rnk = rada
combined.head(20)

,Team,Unnamed: 0,Rnk,Rider,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
11,58.0,18043,1,80.0,29.0,250.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
1,7.0,204503,2,1270.0,34.0,190.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3,19.0,10864,3,55.0,30.0,160.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
0,1.0,31378,4,140.0,29.0,140.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
25,19.0,101165,5,491.0,24.0,120.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
15,75.0,41516,6,190.0,27.0,110.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
18,95.0,26447,7,121.0,28.0,100.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
19,103.0,120903,8,582.0,25.0,90.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
5,28.0,113956,9,552.0,24.0,80.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0
10,52.0,114485,10,554.0,28.0,70.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,0,350,0


In [6]:
result = rate(list(combined.old_elo), list(combined.old_rd))
combined.new_elo = result[0]
combined.new_rd = result[1]

In [7]:
combined.head(10)

,Team,Unnamed: 0,Rnk,Rider,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
11,58.0,18043,1,80.0,29.0,250.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2595.844959,350,207.536485
1,7.0,204503,2,1270.0,34.0,190.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2544.875426,350,207.536485
3,19.0,10864,3,55.0,30.0,160.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2493.905893,350,207.536485
0,1.0,31378,4,140.0,29.0,140.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2442.936360,350,207.536485
25,19.0,101165,5,491.0,24.0,120.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2391.966827,350,207.536485
15,75.0,41516,6,190.0,27.0,110.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2340.997294,350,207.536485
18,95.0,26447,7,121.0,28.0,100.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2290.027761,350,207.536485
19,103.0,120903,8,582.0,25.0,90.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2239.058228,350,207.536485
5,28.0,113956,9,552.0,24.0,80.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2188.088695,350,207.536485
10,52.0,114485,10,554.0,28.0,70.0,11.00,-1.0,3/12/2017,-,0.0,0 km,4,2017,3,12,1500,2137.119162,350,207.536485


In [8]:
dat['Date'] =  pd.to_datetime(dat['Date'])
dat[(dat['Stage'] == -1) & (dat['Name'] == "11.00") & (dat['Date'] >= pd.Timestamp(2017,3,10))].head(10)

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,ProfileScore,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd
2082,18043,8.0,80.0,58.0,29.0,250.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
29568,204503,1.0,1270.0,7.0,34.0,190.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
1274,10864,3.0,55.0,19.0,30.0,160.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3946,31378,49.0,140.0,1.0,29.0,140.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
13967,101165,41.0,491.0,19.0,24.0,120.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
5293,41516,5.0,190.0,75.0,27.0,110.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
3213,26447,42.0,121.0,95.0,28.0,100.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16859,120903,47.0,582.0,103.0,25.0,90.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
15937,113956,32.0,552.0,28.0,24.0,80.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0
16013,114485,178.0,554.0,52.0,28.0,70.0,11.00,-1.0,2017-03-12,-,0.0,0 km,4,2017,3,12,1500,0,350,0


In [9]:
combined.Year[0]

2017

In [10]:
# CREATING THE LOOP
# 1. TAKE ONLY STAGE == -1
# 2. sort by date
# 3. rotate over all names - for name in table.name.unique() - make sure it's taking it in the correct order
# 4. take subset (1 race), compute new elo
# 5. update elos in the central table

df2 = df[(df['Stage'] == -1) & (df['Year'] == 2017)] # df is already only 2017 for now
df2 = df2.sort_values('Date', ascending = True)
df2['old_elo'] = 1500
df2['new_elo'] = 0
df2['old_rd'] = 350
df2['new_rd'] = 0
date1 = []
date = list(df2.Date)
for i in range(len(df2.Date)):
    date1.append(pd.Timestamp(date[i]))
df2['Date1'] = 0
df2['Date1'] = date1

# CREATE A LOOP EVALUATING EVERY RACE
for j in range(len(df2.Name.unique())):
    race_name = list(df2.sort_values('Date',  ascending = True).Name.unique())[j]
    race_data = df2[(df2['Name'] == race_name)]
    # take only two best riders!!!!
    # FILTER 2 BEST RANKED RIDERS
    #first = race_data.groupby('Team').nth(0)
    #second = race_data.groupby('Team').nth(1)

    #combined = first.append(second)
    #combined = combined.reset_index().sort_values('Pnt', ascending=False)
    #rada = np.arange(1, (len(combined.Rnk)+1))
    #combined.Rnk = rada
    #race_data = combined
    result = rate(list(race_data.old_elo), list(race_data.old_rd)) # compute new elos

# create a empty df with all the cols we want
    #empty = pd.DataFrame(columns = df2.columns)

# UPDATING OF VALUES, do after every race, has to be intended
    for i in range(len(race_data.Rider)): 
        rider = list(race_data.Rider)[i]
        elo = result[0][i]
        rd = result[1][i]
        date = pd.Timestamp(list(race_data.Year)[0], list(race_data.Month)[0], list(race_data.Day)[0])

        df2.loc[(df2['Rider'] == rider) & (df2['Date1'] > date), 'old_elo'] = elo
        df2.loc[(df2['Rider'] == rider) & (df2['Date1'] > date), 'old_rd'] = rd
        df2.loc[(df2['Rider'] == rider) & (df2['Date1'] == date), 'new_elo'] = elo
        df2.loc[(df2['Rider'] == rider) & (df2['Date1'] == date), 'new_rd'] = rd



     #   subset1 = df2[(df2['Rider'] == rider) & (df2['Date1'] > date)] # slice of df2 for every rider after this race
      #  subset1.loc[:,'old_elo']  = elo
       # subset1.loc[:,'old_rd']  = rd

        #subset2 = df2[(df2['Rider'] == rider) & (df2['Date1'] == date)] # slice of df2 for every rider after this race
        #subset2.loc[:,'new_elo']  = elo
        #subset2.loc[:,'new_rd']  = rd

        #subset3 = df2[(df2['Rider'] == rider) & (df2['Date1'] < date)]
        # and merge them all together afterwards (append), then use this df for next race
        #empty = empty.append(subset1)
        #empty = empty.append(subset2)
        #empty = empty.append(subset3)

        # HERE APPEND sets_merged TO EMPTY

    #df2 = empty

In [11]:
df2[(df2['Rider'] == 1270)]

,Unnamed: 0,Rnk,Rider,Team,Age,Pnt,Name,Stage,Date,Avg. speed winner,...,Distance,RaceRanking,Year,Month,Day,old_elo,new_elo,old_rd,new_rd,Date1
29558,204493,14.0,1270.0,7.0,34.0,20.0,22.00,-1.0,2/26/2017,-,...,0 km,20,2017,2,26,1500.000000,2571.055241,350.000000,133.346798,2017-02-26
29568,204503,1.0,1270.0,7.0,34.0,190.0,11.00,-1.0,3/12/2017,-,...,0 km,4,2017,3,12,2571.055241,2864.263664,133.346798,110.368405,2017-03-12
29575,204510,5.0,1270.0,7.0,34.0,190.0,1.00,-1.0,3/26/2017,-,...,0 km,18,2017,3,26,2864.263664,1578.553468,110.368405,102.098633,2017-03-26
29582,204517,61.0,1270.0,7.0,34.0,190.0,66.00,-1.0,4/8/2017,-,...,0 km,24,2017,4,8,1578.553468,209.402914,102.098633,87.652012,2017-04-08
29590,204525,70.0,1270.0,7.0,34.0,60.0,5.00,-1.0,6/11/2017,-,...,0 km,6,2017,6,11,209.402914,-56.321344,87.652012,54.756562,2017-06-11
29612,204547,97.0,1270.0,7.0,34.0,210.0,6.00,-1.0,7/23/2017,40.997 km/h,...,0 km,1,2017,7,23,-56.321344,-235.114062,54.756562,37.880456,2017-07-23
29634,204569,4.0,1270.0,7.0,34.0,200.0,8.00,-1.0,9/10/2017,-,...,0 km,16,2017,9,10,-235.114062,37.792518,37.880456,31.665570,2017-09-10


In [12]:
date1 = []
date = list(df2.Date)
for i in range(len(df2.Date)):
    date1.append(pd.Timestamp(date[i]))
df2['Date1'] = 0
df2['Date1'] = date1

In [13]:
subset2

NameError: name 'subset2' is not defined

In [14]:
len(list(df2.Name.unique()))

17

In [16]:
result[0][1]

5094.063219836233